# #7. Views & Co

1. [Views](#Views)
    1. [FBV](#FBV)
    2. [CBV](#CBV)
    3. [CBGV](#CBGV)
2. [Forms](#Forms)
    1. [Basic](#Basic)
    2. [ModelForm](#ModelForm)
    3. [CBGV and Forms](#CBGV-and-Forms)
4. [Templates](#Templates)
    1. [Django template language](#Django-template-language)
    2. [Architecture](#Architecture)
    1. [Filters](#Filters)
    2. [Tags](#Tags)
    4. [Jinja2](#Jinja2)

![cycle1](https://dl.dropboxusercontent.com/s/gzxgfwqjhwogcpg/views1.png)

![cycle2](https://dl.dropboxusercontent.com/s/femnebxbv5dtduk/views2.png)

In [ ]:
import requests
from django.conf import settings

class StackOverflowMiddleware(object):
    def process_exception(self, request, exception):
        if settings.DEBUG:
            intitle = u'{}: {}'.format(exception.__class__.__name__,  exception.message)
            print intitle

            url = 'https://api.stackexchange.com/2.2/search'
            headers = { 'User-Agent': 'github.com/vitorfs/seot' }
            params = {
                'order': 'desc',
                'sort': 'votes',
                'site': 'stackoverflow',
                'pagesize': 3,
                'tagged': 'python;django',
                'intitle': intitle
            }

            r = requests.get(url, params=params, headers=headers)
            questions = r.json()

            print ''

            for question in questions['items']:
                print question['title']
                print question['link']
                print ''

        return None

## Views

![cycle3](https://dl.dropboxusercontent.com/s/jvu3y87uwqlmw0u/views3.png)

__Workflow__
* Accept HttpRequest object
* Generate data based on:
    * HTTP request method
    * Request data (POST, PUT)
    * Data from url dispatch
* Return HttpResponse object

__Whats a view__
* any Python callable
    * functions
        * historically first 
        * easy and simple
        * cannot reuse code
    * classes
        * more generic
        * couldn't handle complex cases
* takes in only request plus some stuff

In [ ]:
from django.http import HttpResponse
from django.views.generic import View

# The simplest FBV
def simplest_view(request):
    # Logic goes here
    return HttpResponse('FBV')

# The simplest CBV
class SimplestView(View):
    def get(self, request, *args, **kwargs):
        # Logic goes here
        return HttpResponse('CBV')

#### FBV

* less view code is better
* DRY
* views should handle presentation logic
    * keep business logic in models
    * or in forms
* use them for custom 403, 404 and 500 handlers

In [ ]:
from decorator_plus import require_http_methods
from django.shortcuts import get_object_or_404, redirect, render

from .forms import ExampleForm
from .models import ExampleModel


@require_http_methods(['GET'])
def model_detail(request, *args, **kwargs):
    request_slug = kwargs.get('slug')
    example_obj = get_object_or_404(
        ExampleModel, slug=request_slug)
    return render(
        request,
        'viewsapp/detail.html',
        {'object': example_obj})


@require_http_methods(['GET', 'POST'])
def model_create(request, *args, **kwargs):
    if request.method == 'POST':
        form = ExampleForm(request.POST)
        if form.is_valid():
            new_obj = form.save()
            return redirect(new_obj)
    else:
        form = ExampleForm()
    return render(request, 'viewsapp/form.html', {'form': form})

Code reuse accomplshed by using utils module (or core app)

In [ ]:
# slots/utils.py
from django.core.exceptions import PermissionDenied


# it's handy to pass request to utils functions
def check_slot_rights(request):
    if request.user.is_staff:
        request.allowed_slot = True
        return request

    # Return a HTTP 403 back to the user
    raise PermissionDenied

In [ ]:
# slots/views.py
from django.shortcuts import get_object_or_404
from django.shortcuts import render
from django.views.decorators.http import require_http_methods

from .models import Slot
from .utils import check_slot_rights

@require_http_methods(["GET", "HEAD"])
def slot_list(request):
    """Standard list view"""

    request = check_slot_rights(request)

    return render(request,
        "slots/slot_list.html",
        {"slots": Slot.objects.all()})

@require_safe
def slot_detail(request, pk):
    """Standard detail view"""

    request = check_slot_rights(request)

    slot = get_object_or_404(Slot, pk=pk)

    return render(request, "slots/slot_detail.html",
        {"slot": slot})

@require_safe
def slot_preview(request):
    slot = Slot.objects.all()
    return render(request,
        "slots/slot_preview.html",
        {"slot": slot})

Or use decorators

In [ ]:
# slots/decorators.py
from functools import wraps

from . import utils

# based off the decorator template from the previous chapter
def check_slots(view_func):
    """Check if a user can add slots"""
    @wraps(view_func)
    def new_view_func(request, *args, **kwargs):
        request = utils.check_slot_rights(request)

        # Call the view function
        response = view_func(request, *args, **kwargs)

        # Return the HttpResponse object
        return response
    return new_view_func

In [ ]:
# slots/views.py
from django.shortcuts import get_object_or_404, render

from .decorators import check_slots
from .models import Slot

@require_safe
@check_slots
def slot_detail(request, pk):
    """Standard detail view"""

    slot = get_object_or_404(Slot, pk=pk)

    return render(request, "slots/slot_detail.html",
        {"slot": slot})

#### CBV

Main schools of thought
* use all generic views
* use django.views.generic.View
* use only when subclassing is needed

In [ ]:
from django.shortcuts import (
    get_object_or_404, redirect, render)
from django.views.generic import View

from .forms import ExampleForm
from .models import ExampleModel


class ModelDetail(View):

    def get(self, request, *args, **kwargs):
        request_slug = kwargs.get('slug')
        example_obj = get_object_or_404(
            ExampleModel, slug=request_slug)
        return render(
            request,
            'viewsapp/detail.html',
            {'object': example_obj})


class ModelCreate(View):
    context_object_name = 'form'
    form_class = ExampleForm
    template_name = 'viewsapp/form.html'

    def get(self, request, *args, **kwargs):
        return render(
            request,
            self.template_name,
            {self.context_object_name:
                self.form_class()})

    def post(self, request, *args, **kwargs):
        bound_form = self.form_class(request.POST)
        if bound_form.is_valid():
            new_obj = bound_form.save()
            return redirect(new_obj)
        return render(
            request,
            self.template_name,
            {self.context_object_name:
                bound_form})

Out of the box
* HEAD handling
* OPTIONS handling
* clearer (sometimes more easy to read)

#### CBGV

In [ ]:
from django.views.generic import CreateView, DetailView

from .forms import ExampleForm
from .models import ExampleModel


class ModelDetail(DetailView):
    model = ExampleModel
    template_name = 'viewsapp/detail.html'


class ModelCreate(CreateView):
    context_object_name = 'form'
    form_class = ExampleForm
    template_name = 'viewsapp/form.html'

Example CBGV namespaced

In [ ]:
from django.urls import reverse
from django.views.generic import ListView, DetailView, UpdateView

from .models import Tasting

class TasteListView(ListView):
    model = Tasting

class TasteDetailView(DetailView):
    model = Tasting

class TasteResultsView(TasteDetailView):
    template_name = 'tastings/results.html'

class TasteUpdateView(UpdateView):
    model = Tasting

    def get_success_url(self):
        return reverse('tastings:detail',
            kwargs={'pk': self.object.pk})

In [ ]:
# tastings/urls.py
from django.conf.urls import url

from . import views

urlpatterns = [
    url(
        regex=r'^$',
        view=views.TasteListView.as_view(),
        name='list'
    ),
    url(
        regex=r'^(?P<pk>\d+)/$',
        view=views.TasteDetailView.as_view(),
        name='detail'
    ),
    url(
        regex=r'^(?P<pk>\d+)/results/$',
        view=views.TasteResultsView.as_view(),
        name='results'
    ),
    url(
        regex=r'^(?P<pk>\d+)/update/$',
        view=views.TasteUpdateView.as_view(),
        name='update'
    )
]

In [ ]:
# urls.py
urlpatterns += [
    url(r'^tastings/', include('tastings.urls', namespace='tastings')),
]

![hierra](http://andrewsforge.com/images/otwcbv/5_rank_simple_color.png)

![gcbv](https://cwza.github.io/my_blog/2016/02/19/django%E5%AD%B8%E7%BF%92%E7%AD%86%E8%A8%98/table10_1.png)

Example view object properties

In [ ]:
class FavoriteMixin(object):

    @cached_property
    def likes_and_favorites(self):
        """Returns a dictionary of likes and favorites"""
        likes = self.object.likes()
        favorites = self.object.favorites()
        return {
            "likes": likes,
            "favorites": favorites,
            "favorites_count": favorites.count(),

        }

class FlavorUpdateView(LoginRequiredMixin, FavoriteMixin, UpdateView):
    model = Flavor
    fields = ['title', 'slug', 'scoops_remaining']

class FlavorDetailView(LoginRequiredMixin, FavoriteMixin, TemplateView):
    model = Flavor

In [ ]:
{# flavors/base.html #}
{% extends "base.html" %}

{% block likes_and_favorites %}
<ul>
  <li>Likes: {{ view.likes_and_favorites.likes }}</li>
  <li>Favorites: {{ view.likes_and_favorites.favorites_count }}</li>
</ul>
{% endblock likes_and_favorites %}

### References

* http://www.b-list.org/weblog/2006/jun/13/how-django-processes-request/
* https://docs.djangoproject.com/en/1.11/topics/http/views/
* https://docs.djangoproject.com/en/1.11/topics/class-based-views/intro/
* http://ccbv.co.uk/
* https://www.youtube.com/watch?v=BJiOERA49ZQ

### Summary

![views](http://3.bp.blogspot.com/-94MWJfD92j4/UnC19NyaThI/AAAAAAAAC_I/422GeSIQceo/s1600/00010.jpeg)

## Forms

![form](https://mdn.mozillademos.org/files/14205/Form%20Handling%20-%20Standard.png)

#### Basic

In [ ]:
from django.utils.translation import gettext_lazy as _
from django import forms

def require_pony(value):
    if "pony" not in value:
        raise forms.ValidationError("Value must contain a pony" ) 

class ContactForm(forms.Form):

    name = forms.CharField(label=_("Your Name"),
        max_length=255,
        widget=forms.TextInput,
    )
    email = forms.EmailField(label=_("Email address"))
    pony = forms.CharField(validators=[require_pony])

__Widgets__

* One for each type of HTML form control
* Low-level operations
* Know how to render appropriate HTML
* Know how to pull their data out of a submission
* Can bundle arbitrary media (CSS, JavaScript) to include and use when rendering

I/O
* When data is submitted, a widget’s
`value_from_datadict()` method pulls out that
widget’s value
* When displaying a form, a widget’s `render()`
method is responsible for generating that widget’s
HTML

__Fields__

* Represent data type and validation constraints
* Have widgets associated with them for rendering

#### Validation

* Call the form’s `is_valid()` method that calls `full_clean()`
* That applies field-level validation
    * For each field, call its widget’s `value_from_datadict()` to get the value for the field
    * Call field’s `clean()` method
    * This calls field’s `to_python()` method to convert value to correct Python type
    * Then calls field’s `validate()` method to run field’s own built-in validation
    * Then calls field’s `run_validators()` method to run any additional validators supplied by end user
    * `clean()` either returns a valid value, or raises ValidationError
* Then form-level validation
    * Form’s `clean()` method
        * Call any `clean_*fieldname*()` method on the form (if field `clean()` ran without errors)
        * Errors go into form’s errors dict, keyed by field name
    * Happens after field validation, so cleaned_data may or may not be populated depending on prior errors
    * Errors raised here end up in error dict under the key \__all\__, or by calling `non_field_errors()`
* Then sets up either cleaned_data or errors attribute

Example Form workflow

In [ ]:
from django.shortcuts import get_object_or_404
from django.http import HttpResponseRedirect
from django.core.urlresolvers import reverse
import datetime

from .forms import RenewBookForm

def renew_book_librarian(request, pk):
    book_inst=get_object_or_404(BookInstance, pk = pk)

    # If this is a POST request then process the Form data
    if request.method == 'POST':

        # Create a form instance and populate it with data from the request (binding):
        form = RenewBookForm(request.POST)

        # Check if the form is valid:
        if form.is_valid():
            # process the data in form.cleaned_data as required (here we just write it to the model due_back field)
            book_inst.due_back = form.cleaned_data['renewal_date']
            book_inst.save()

            # redirect to a new URL:
            return HttpResponseRedirect(reverse('all-borrowed') )

    # If this is a GET (or any other method) create the default form.
    else:
        proposed_renewal_date = datetime.date.today() + datetime.timedelta(weeks=3)
        form = RenewBookForm(initial={'renewal_date': proposed_renewal_date,})

    return render(request, 'catalog/book_renew_librarian.html', {'form': form, 'bookinst':book_inst})

In [ ]:
from django import forms

from django.core.exceptions import ValidationError
from django.utils.translation import ugettext_lazy as _
import datetime
    
class RenewBookForm(forms.Form):
    renewal_date = forms.DateField(help_text="Enter a date between now and 4 weeks (default 3).")

    def clean_renewal_date(self):
        data = self.cleaned_data['renewal_date']
        
        #Check date is not in past. 
        if data < datetime.date.today():
            raise ValidationError(_('Invalid date - renewal in past'))

        #Check date is in range librarian allowed to change (+4 weeks).
        if data > datetime.date.today() + datetime.timedelta(weeks=4):
            raise ValidationError(_('Invalid date - renewal more than 4 weeks ahead'))

        # Remember to always return the cleaned data.
        return data

Example custom Field and validation

In [ ]:
from django import forms
from django.core.validators import validate_email

class MultiEmailField(forms.Field):
    def to_python(self, value):
        """Normalize data to a list of strings."""
        # Return an empty list if no input was given.
        if not value:
            return []
        return value.split(',')

    def validate(self, value):
        """Check if value consists only of valid emails."""
        # Use the parent's handling of required fields, etc.
        super(MultiEmailField, self).validate(value)
        for email in value:
            validate_email(email)

In [ ]:
class ContactForm(forms.Form):
    subject = forms.CharField(max_length=100)
    message = forms.CharField()
    sender = forms.EmailField()
    recipients = MultiEmailField()
    cc_myself = forms.BooleanField(required=False)

    def clean_recipients(self):
        data = self.cleaned_data['recipients']
        if "fred@example.com" not in data:
            raise forms.ValidationError("You have forgotten about Fred!")

        # Always return a value to use as the new cleaned data, even if
        # this method didn't change it.
        return data

    def clean(self):
        cleaned_data = super(ContactForm, self).clean()
        cc_myself = cleaned_data.get("cc_myself")
        subject = cleaned_data.get("subject")

        if cc_myself and subject:
            # Only do something if both fields are valid so far.
            if "help" not in subject:
                raise forms.ValidationError(
                    "Did not send for 'help' in the subject despite "
                    "CC'ing yourself."
                )

__Build Form dynamically__

* Forms have metaclass
    * can use it directly if you want: django.forms.BaseForm
    * but probably not a good idea unless you’re constructing form classes dynamically at runtime
* form class ends up with two field-related attributes: base_fields and fields.
    * base_fields is the default set of fields for all instances of that form class.
    * fields is the set of fields for a specific instance of the form class.

In [ ]:
base_fields = {
    "name": forms.CharField(max_length=255),
    "email": forms.EmailField(),
    "message": forms.CharField(widget=forms.Textarea),
}

ContactForm = type("ContactForm",
    (forms.BaseForm,),
    {"base_fields": base_fields}
)

In [ ]:
class ContactForm(forms.Form):
    name = forms.CharField(max_length=255)
    email = forms.EmailField()
    message = forms.CharField(widget=forms.Textarea)

#### ModelForm

In [ ]:
#in models.py
class Profile(models.Model):
    user = models.OneToOneField(User)
    company = models.CharField(max_length=50)

#in forms.py
class ProfileForm(forms.ModelForm):
    def __init__(self, *args, **kwargs):
        super(ProfileForm, self).__init__(*args, **kwargs)
        self.fields['company'].validators.append(require_pony)
        self.fields['company'].required = True

    class Meta:
        model = Profile
        # never use exclude
        fields = ['company',]

#In views.py:
form = ProfileForm(request.POST)
profile = form.save(commit = False)
profile.user = request.user
profile.save()

* Introspects a model class and makes a form out of it
* Basic structure (declarative class plus inner options class) is similar to models
* Uses model meta API to get lists of fields, etc.
* Override/configure things by setting options in Meta

* Calls the `formfield()` method of each field in the model
* Can be overridden by defining the method formfield_callback() on the form
* Each field’s `value_from_object()` method called to get the value of that field on that model instance

__Saving__
* django.forms.models.construct_instance() builds an instance of the model with form data filled in
* django.forms.models.save_instance() actually (if commit=True) saves the instance to the DB
* Saving of many-to-many relations is deferred until after the instance itself has been saved
    * with commit=False, you have to manually do it

#### CBGV and Forms

In [ ]:
# flavors/views.py
from django.contrib import messages
from django.contrib.auth.mixins import LoginRequiredMixin
from django.views.generic import CreateView, DetailView, UpdateView

from .models import Flavor

class FlavorActionMixin:

    fields = ['title', 'slug', 'scoops_remaining']

    @property
    def success_msg(self):
        return NotImplemented

    def form_valid(self, form):
        messages.info(self.request, self.success_msg)
        return super(FlavorActionMixin, self).form_valid(form)

class FlavorCreateView(LoginRequiredMixin, FlavorActionMixin,
                        CreateView):
    model = Flavor
    success_msg = "Flavor created!"

class FlavorUpdateView(LoginRequiredMixin, FlavorActionMixin,
                        UpdateView):
    model = Flavor
    success_msg = "Flavor updated!"

class FlavorDetailView(DetailView):
    model = Flavor

In [ ]:
{% if messages %}
    <ul class="messages">
        {% for message in messages %}
        <li id="message_{{ forloop.counter }}"
            {% if message.tags %} class="{{ message.tags }}"
                {% endif %}>
            {{ message }}
        </li>
        {% endfor %}
    </ul>
{% endif %}

Example 2 Forms, 1 Model

In [ ]:
# stores/models.py
from django.db import models
from django.urls import reverse

class IceCreamStore(models.Model):
    title = models.CharField(max_length=100)
    block_address = models.TextField()
    phone = models.CharField(max_length=20, blank=True)
    description = models.TextField(blank=True)

    def get_absolute_url(self):
        return reverse('store_detail', kwargs={'pk': self.pk})

In [ ]:
# stores/forms.py
from django import forms

from .models import IceCreamStore

class IceCreamStoreCreateForm(forms.ModelForm):

    class Meta:
        model = IceCreamStore
        fields = ['title', 'block_address', ]

class IceCreamStoreUpdateForm(IceCreamStoreCreateForm):

    def __init__(self, *args, **kwargs):
        super(IceCreamStoreUpdateForm,
                self).__init__(*args, **kwargs)
        self.fields['phone'].required = True
        self.fields['description'].required = True

    class Meta(IceCreamStoreCreateForm.Meta):
        # show all the fields!
        fields = ['title', 'block_address', 'phone',
                'description', ]

In [ ]:
# stores/views
from django.views.generic import CreateView, UpdateView

from .forms import IceCreamStoreCreateForm, IceCreamStoreUpdateForm
from .models import IceCreamStore

class IceCreamCreateView(CreateView):
    model = IceCreamStore
    form_class = IceCreamStoreCreateForm

class IceCreamUpdateView(UpdateView):
    model = IceCreamStore
    form_class = IceCreamStoreUpdateForm

Example using Form for CSV (not only POST)

In [ ]:
import csv

from django.utils.six import StringIO

from django import forms

from .models import Purchase, Seller

class PurchaseForm(forms.ModelForm):

    class Meta:
        model = Purchase

    def clean_seller(self):
        seller = self.cleaned_data['seller']
        try:
            Seller.objects.get(name=seller)
        except Seller.DoesNotExist:
            msg = '{0} does not exist in purchase #{1}.'.format(
                seller,
                self.cleaned_data['purchase_number']
            )
            raise forms.ValidationError(msg)
        return seller

def add_csv_purchases(rows):

    rows = StringIO.StringIO(rows)

    records_added = 0
    errors = []
    # Generate a dict per row, with the first CSV row being the keys.
    for row in csv.DictReader(rows, delimiter=','):

        # Bind the row data to the PurchaseForm.
        form = PurchaseForm(row)
        # Check to see if the row data is valid.
        if form.is_valid():
            # Row data is valid so save the record.
            form.save()
            records_added += 1
        else:
            errors.append(form.errors)

    return records_added, errors

Example adding additional attribute to form

In [ ]:
from django import forms

from .models import Taster

class TasterForm(forms.ModelForm):

    class Meta:
        model = Taster

    def __init__(self, *args, **kwargs):
        self.user = kwargs.pop('user')
        super(TasterForm, self).__init__(*args, **kwargs)

In [ ]:

from django.contrib.auth.mixins import LoginRequiredMixin
from django.views.generic import UpdateView

from .forms import TasterForm
from .models import Taster

class TasterUpdateView(LoginRequiredMixin, UpdateView):
    model = Taster
    form_class = TasterForm
    success_url = '/someplace/'

    def get_form_kwargs(self):
        """This method is what injects forms with keyword arguments."""
        # grab the current set of form #kwargs
        kwargs = super(TasterUpdateView, self).get_form_kwargs()
        # Update the kwargs with the user_id
        kwargs['user'] = self.request.user
        return kwargs

### References

* https://developer.mozilla.org/en-US/docs/Learn/Server-side/Django/Forms
* https://www.pydanny.com/tag/forms.html
* https://bradmontgomery.net/blog/2015/04/25/nice-arrayfield-widgets-choices-and-chosenjs/
* https://docs.djangoproject.com/en/1.11/ref/forms/renderers/

### Summary

* use ModelForm if you can
* know validation process
* creating custom widget - keep it simple

## Templates

* Separate logic from presentation
* Discourage redundancy
    * inheritance
* Be decoupled from HTML
* XML should not be used for template languages
* Assume designer competence
* Treat whitespace obviously
    * Any whitespace that’s not in a template tag should be displayed.
* Don’t invent a programming language
    * The Django Template Language (DTL) aims to avoid advanced logic.
* Safety and security
* Extensibility

#### Django template language

* Engine
    * New in Django 1.8
    * Wrap a template system in a consistent API
    * Built-in engines for Django’s template language and for Jinja2
* Loader
    * Required method: `load_template_source()`, taking template name and optional directories to look in
    * Should return a 2-tuple of template contents and template location
    * Could load templates from database or cache
* Template
    * Instantiate with template source
    * Implement `render()` method receiving optional context (dictionary) and optional request (HttpRequest object), returning string
* Tag and filter libraries
* Context

Key parts
* All in django/template/base.py
* Template is the high-level representation
* Lexer and Parser actually turn the template source into something usable
* Token represents bits of the template during parsing
* Node and NodeList are the structures which make up a compiled Template

Lexing
* Instantiate with template source, then call `tokenize()`
* Splits the template source using a regex which recognizes start/end syntax of tags and variables
* Creates and returns a list of tokens based on the resulting substrings

Tokens
* Come in four flavors: Text, Var, Block, Comment
* Gets the text of some piece of template syntax, minus the start/end constructs like {% … %} or {{ … }}

Parsing
* Instantiate Parser with list of tokens from Lexer, then call parse()
* Each tag and filter (built-in or custom) gets registered with Parser
* Each tag provides a compilation function, which Parser will call to produce a Node
* Variables get automatically translated into VariableNode, plain text into TextNode

Nodes
* Can be an individual Node, or a NodeList
* Defining characteristic is the render() method which takes a Context as argument and returns a string

Workflow
* transform the string source of the template into a NodeList
* one Node for each tag, each variable and each bit of plain text.
* rendering involves having that NodeList iterate over and render() its Nodes
* concatenate the results into a single string which is the output.

Context
* Lives in django/template/context.py
* Behaves like a dictionary
* Is actually a stack of dictionaries, supporting push/pop and fall-through lookups
* First dictionary in the stack to return a value wins

#### Architecture

__2-tier__

Best for sites where layout is consistent

__3-tier__

__Inheritance__

In [ ]:
{# simple base.html #}
{% load staticfiles %}
<html>
<head>
    <title>
        {% block title %}Two Scoops of Django{% endblock title %}
    </title>
    {% block stylesheets %}
        <link rel="stylesheet" type="text/css" href="{% static 'css/project.css' %}">
    {% endblock stylesheets %}
</head>
<body>
    <div class="content">
        {% block content %}
            <h1>Two Scoops</h1>
        {% endblock content %}
    </div>
</body>
</html>

In [ ]:
{% extends "base.html" %}
{% load staticfiles %}
{% block title %}About Audrey and Daniel{% endblock title %}
{% block stylesheets %}
    {{ block.super }}
    <link rel="stylesheet" type="text/css" href="{% static 'css/about.css' %}">
{% endblock stylesheets %}
{% block content %}
    {{ block.super }}
    <h2>About Audrey and Daniel</h2>
    <p>They enjoy eating ice cream</p>
{% endblock content %}

__Debug__

In [ ]:
# settings/local.py
TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        'APP_DIRS': True,
        'OPTIONS':
        {
            'string_if_invalid': 'INVALID EXPRESSION: %s'
        },
    },
]

#### Filters

In [ ]:
@register.filter(is_safe=False)
def length(value):
    """Returns the length of the value - useful for lists."""
    try:
        return len(value)
    except (ValueError, TypeError):
        return 0

Example custom filter

In [ ]:
from django import template
from datetime import date, timedelta

register = template.Library()

@register.filter(name='get_due_date_string')
def get_due_date_string(value):
    delta = value - date.today()

    if delta.days == 0:
        return "Today!"
    elif delta.days < 1:
        return "%s %s ago!" % (abs(delta.days),
            ("day" if abs(delta.days) == 1 else "days"))
    elif delta.days == 1:
        return "Tomorrow"
    elif delta.days > 1:
        return "In %s days" % delta.days

In [ ]:
{% load app_filters %}
{% for todo in todos %}
    ...
    <div class='due_date'>
        <b>Due:</b> {{ todo.due_date|get_due_date_string }}
    </div>
    ...
{% endfor %}

#### Tags

* Each tag must provide compilation function
* It gets access to the token representing the tag, and to the parser
* Can just be simple and instantiate/return a Node subclass
* Or can use the parser to do more complex things like parsing ahead to an end tag, doing things with all the nodes in between, etc
* Allows tags to set groups of variables by pushing a new dictionary on top, then clean up by popping
* Performance implications: just as nested namespaces in Python code will slow down name lookups

General
* put custom tags to *app_name*_tags.py
* load them in templates

In [ ]:
{% extends "base.html" %}

{% load flavors_tags %}

Simple tag
* Processes the data and returns a string

In [ ]:
from django import template
register = template.Library()

from ..models import YourModel

@register.simple_tag
def any_function():
    return YourModel.objects.count()

Inclusion tag
* Processes the data and returns a rendered template

In [ ]:
@register.inclusion_tag('path_to_your_html_file.html')
def any_function():
    variable = YourModel.objects.order_by('-publish')[:5]
    return {'variable': variable}

Assignment tag
* Processes the data and sets a variable in the contex

In [ ]:
@register.assignment_tag
def any_function(count=5):
    return *some database query*

In [ ]:
{% get any_function as queries %}
<ul>
{% for query in queries %}
  <li>
  ...
  </li>
{% endfor %}
</ul>

Custom tag

In [ ]:
from django import template

@register.tag(name="current_time")
def do_current_time(parser, token):
    try:
        # split_contents() knows not to split quoted strings.
        tag_name, format_string = token.split_contents()
    except ValueError:
        raise template.TemplateSyntaxError(
            "%r tag requires a single argument" % token.contents.split()[0]
        )
    if not (format_string[0] == format_string[-1] and format_string[0] in ('"', "'")):
        raise template.TemplateSyntaxError(
            "%r tag's argument should be in quotes" % tag_name
        )
    return CurrentTimeNode(format_string[1:-1])

In [ ]:
import datetime
from django import template

class CurrentTimeNode(template.Node):
    def __init__(self, format_string):
        self.format_string = format_string

    def render(self, context):
        return datetime.datetime.now().strftime(self.format_string)

#### Jinja2

![diff](http://pylixm.cc/images/df.png)

* can be used independetly
* syntax may be more pleasant
* more explicit
* its faster

### References

* https://github.com/jazzband/django-dbtemplates/blob/master/dbtemplates/loader.py
* https://github.com/django/django/blob/stable/1.11.x/django/template/defaultfilters.py
* https://www.codementor.io/hiteshgarg14/creating-custom-template-tags-in-django-application-58wvmqm5f

### Summary

* know the implications of DTL
* in CBV use object and object_list implicit variables
* template called with include should be prefixed with _
* prefer underscores over dashes in template names
* avoid coupling styles code to Python code. Control styling with css
* use URL names instead of hardcoded paths
* structure HTML nicely
* watch for implicit database quries/rpc calls in templates code
* keep presentation logic mostly out of templates put it yourr views